# 01_preprocessing_eda_v2 — Robust EDA & Preprocessing

In [1]:

# Imports & paths
import os, sys, json, math, random
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

random.seed(42); np.random.seed(42)
plt.rcParams['figure.figsize'] = (9,5); plt.rcParams['axes.grid'] = True

NB_DIR = Path.cwd()
PROJECT_DIR = NB_DIR.parent if NB_DIR.name=='notebooks' else NB_DIR
DATA_RAW = PROJECT_DIR / 'data' / 'raw'
DATA_PROCESSED = PROJECT_DIR / 'data' / 'processed'
for p in [DATA_RAW, DATA_PROCESSED]: p.mkdir(parents=True, exist_ok=True)
print('DATA_RAW:', DATA_RAW)


DATA_RAW: c:\Users\byed2\Documents\miacd\Aprendizaje Profundo\Proyecto Final\kan_mlp_sales\data\raw


In [2]:

# Load raw files (train, features, stores)
train_p = DATA_RAW / 'train.csv'
feat_p  = DATA_RAW / 'features.csv'
stores_p= DATA_RAW / 'stores.csv'
assert train_p.exists(), f'Missing {train_p}'
assert feat_p.exists(), f'Missing {feat_p}'
assert stores_p.exists(), f'Missing {stores_p}'

train = pd.read_csv(train_p, parse_dates=['Date'])
features = pd.read_csv(feat_p, parse_dates=['Date'])
stores = pd.read_csv(stores_p)
print('Loaded shapes:', train.shape, features.shape, stores.shape)


Loaded shapes: (421570, 5) (8190, 12) (45, 3)


In [3]:

# Merge & basic temporal features
df = train.merge(stores, on='Store', how='left').merge(features, on=['Store','Date','IsHoliday'], how='left')
df = df.sort_values(['Store','Dept','Date']).reset_index(drop=True)
df['IsHoliday'] = df['IsHoliday'].astype(int)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
df['Quarter'] = df['Date'].dt.quarter
print('Merged df shape:', df.shape, 'Date range:', df['Date'].min(), '->', df['Date'].max())


Merged df shape: (421570, 20) Date range: 2010-02-05 00:00:00 -> 2012-10-26 00:00:00


In [4]:

# MarkDowns: imputar 0 y flags
md_cols = [c for c in ['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'] if c in df.columns]
for c in md_cols:
    df[c+'_missing'] = df[c].isna().astype(int)
    df[c] = df[c].fillna(0.0)
print('MarkDown cols:', md_cols)


MarkDown cols: ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']


In [5]:

# Calendar sin/cos and holiday proximity by Store-Dept
import numpy as np
df['sin_week'] = np.sin(2*np.pi*df['Week']/52.0)
df['cos_week'] = np.cos(2*np.pi*df['Week']/52.0)
df['sin_month'] = np.sin(2*np.pi*df['Month']/12.0)
df['cos_month'] = np.cos(2*np.pi*df['Month']/12.0)

# Prev/Next holiday flags per Store-Dept
df['IsHolidayPrevWeek'] = df.groupby(['Store','Dept'])['IsHoliday'].shift(1).fillna(0).astype(int)
df['IsHolidayNextWeek'] = df.groupby(['Store','Dept'])['IsHoliday'].shift(-1).fillna(0).astype(int)


In [6]:

# Lags y rolling (basados en valores pasados): lag_1, lag_2, lag_52, roll_mean_4, roll_mean_13
grp = df.groupby(['Store','Dept'], group_keys=False)
df['lag_1'] = grp['Weekly_Sales'].shift(1)
df['lag_2'] = grp['Weekly_Sales'].shift(2)
df['lag_52'] = grp['Weekly_Sales'].shift(52)

# Para los rolling, aplicar por grupo y asignar directamente para evitar problemas de índices
df['roll_mean_4'] = grp['Weekly_Sales'].shift(1).groupby([df['Store'], df['Dept']]).rolling(4, min_periods=1).mean().reset_index(level=[0,1], drop=True)
df['roll_mean_13'] = grp['Weekly_Sales'].shift(1).groupby([df['Store'], df['Dept']]).rolling(13, min_periods=1).mean().reset_index(level=[0,1], drop=True)

# Alternativamente, para mayor robustez y claridad, se puede usar transform:
# df['roll_mean_4'] = grp['Weekly_Sales'].shift(1).groupby([df['Store'], df['Dept']]).transform(lambda x: x.rolling(4, min_periods=1).mean())
# df['roll_mean_13'] = grp['Weekly_Sales'].shift(1).groupby([df['Store'], df['Dept']]).transform(lambda x: x.rolling(13, min_periods=1).mean())

# Flags para lags/rollings faltantes
for c in ['lag_1','lag_2','lag_52','roll_mean_4','roll_mean_13']:
    df[c+'_missing'] = df[c].isna().astype(int)


In [7]:

# Categóricos: one-hot Type, and create store_idx, dept_idx for embeddings
df['Type'] = df['Type'].astype('category')
df = pd.get_dummies(df, columns=['Type'], drop_first=False)
store_map = {v:i for i,v in enumerate(sorted(df['Store'].unique()))}
dept_map  = {v:i for i,v in enumerate(sorted(df['Dept'].unique()))}
df['store_idx'] = df['Store'].map(store_map).astype(int)
df['dept_idx'] = df['Dept'].map(dept_map).astype(int)


In [8]:

# Splits by time (reuse existing metadata if present)
meta_prev = DATA_PROCESSED / 'metadata.json'
if meta_prev.exists():
    import json
    mp = json.load(open(meta_prev))
    train_end = mp.get('train_end'); valid_end = mp.get('valid_end')
    from pandas import to_datetime
    train_end = to_datetime(train_end); valid_end = to_datetime(valid_end)
else:
    dates = np.sort(df['Date'].unique())
    train_end = dates[int(0.8*len(dates))]
    valid_end = dates[int(0.9*len(dates))]
train_mask = df['Date'] <= train_end
valid_mask = (df['Date'] > train_end) & (df['Date'] <= valid_end)
test_mask = df['Date'] > valid_end
df_train = df[train_mask].copy(); df_valid = df[valid_mask].copy(); df_test = df[test_mask].copy()
print('Train/Valid/Test shapes:', df_train.shape, df_valid.shape, df_test.shape)


Train/Valid/Test shapes: (338738, 45) (41369, 45) (41463, 45)


In [9]:

# Selección de características v2 y guardado robusto de imputación+escalado como CSVs v2
target_col = 'Weekly_Sales'
base_cols = ['Store','Dept','Size','Temperature','Fuel_Price','CPI','Unemployment','IsHoliday','Year','Month','Week','Quarter','sin_week','cos_week','sin_month','cos_month','IsHolidayPrevWeek','IsHolidayNextWeek'] + [c for c in df.columns if c.startswith('Type_')]
lag_cols = ['lag_1','lag_2','lag_52','roll_mean_4','roll_mean_13']
md_flag_cols = [c for c in df.columns if c.endswith('_missing') and c.startswith('MarkDown')]
feature_cols_v2 = [c for c in (base_cols + md_flag_cols + lag_cols + [c+'_missing' for c in lag_cols]) if c in df.columns]

# Construir X/y e imputar+escalar usando estadísticas del train
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
imp = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_tr = df_train[feature_cols_v2].copy()
X_va = df_valid[feature_cols_v2].copy()
X_te = df_test[feature_cols_v2].copy()
# Ajustar imputador+escalador en train
X_tr_imp = pd.DataFrame(imp.fit_transform(X_tr), columns=feature_cols_v2, index=X_tr.index)
X_tr_scl = pd.DataFrame(scaler.fit_transform(X_tr_imp), columns=feature_cols_v2, index=X_tr_imp.index)
X_va_scl = pd.DataFrame(scaler.transform(pd.DataFrame(imp.transform(X_va), columns=feature_cols_v2, index=X_va.index)), columns=feature_cols_v2, index=X_va.index)
X_te_scl = pd.DataFrame(scaler.transform(pd.DataFrame(imp.transform(X_te), columns=feature_cols_v2, index=X_te.index)), columns=feature_cols_v2, index=X_te.index)

def save_proc(Xs, ys, path):
    out = Xs.copy()
    out[target_col] = ys.values
    out.to_csv(path, index=False)

save_proc(X_tr_scl, df_train[target_col], DATA_PROCESSED / 'train_processed_v2.csv')
save_proc(X_va_scl, df_valid[target_col], DATA_PROCESSED / 'valid_processed_v2.csv')
save_proc(X_te_scl, df_test[target_col], DATA_PROCESSED / 'test_processed_v2.csv')

# Guardar parquet para embeddings con columnas cont_* y store_idx/dept_idx
cont_cols = [c for c in feature_cols_v2 if not c.startswith('Type_') and c not in ['Store','Dept','store_idx','dept_idx']]
pack_tr = pd.DataFrame({'store_idx': df_train['store_idx'].values, 'dept_idx': df_train['dept_idx'].values, target_col: df_train[target_col].values})

# Solución: aplicar imputador y escalador SOLO sobre las columnas continuas, usando el mismo orden y columnas que en el fit
# Extraer las columnas continuas del set de entrenamiento ya imputado y escalado
cont_scaled_tr = X_tr_scl[cont_cols].copy()
cont_scaled_tr.columns = [f'cont_{i}' for i in range(len(cont_cols))]
cont_scaled_tr.index = df_train.index

pack_tr = pd.concat([pack_tr, cont_scaled_tr], axis=1)
pack_tr['y_log1p'] = np.log1p(pack_tr[target_col])
pack_tr.to_parquet(DATA_PROCESSED / 'train_embeddings_v2.parquet', index=False)

# Guardar metadatos
meta_v2 = {'train_end': str(pd.to_datetime(df_train['Date'].max()).date()), 'valid_end': str(pd.to_datetime(df_valid['Date'].max()).date()), 'feature_cols_v2': feature_cols_v2}
import json
json.dump(meta_v2, open(DATA_PROCESSED / 'metadata_v2.json','w'), indent=2)
print('Guardados los CSVs procesados v2 y el parquet de embeddings.')


c:\Users\byed2\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\byed2\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Guardados los CSVs procesados v2 y el parquet de embeddings.


In [10]:

# Sanity check: ensure no NaNs remain in processed csvs
for p in ['train_processed_v2.csv','valid_processed_v2.csv','test_processed_v2.csv']:
    dfp = pd.read_csv(DATA_PROCESSED / p)
    na = dfp.isna().sum()
    na = na[na>0]
    print(p, 'NaNs count by col:\n', na if not na.empty else 'OK - no NaNs')


train_processed_v2.csv NaNs count by col:
 OK - no NaNs
valid_processed_v2.csv NaNs count by col:
 OK - no NaNs
test_processed_v2.csv NaNs count by col:
 OK - no NaNs
